In [1]:
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
sys.path.append("C:/dev/pycmqlib3/")
sys.path.append("C:/dev/pycmqlib3/misc_scripts/")
warnings.filterwarnings('ignore')
import misc
import data_handler as dh
import plotly_examples
import backtest
import dbaccess
from sqlalchemy import create_engine
%matplotlib inline

ferrous_products_mkts = ['rb', 'hc', 'i', 'j', 'jm']
ferrous_mixed_mkts = ['ru', 'FG', 'ZC', 'SM', "SF"]
base_metal_mkts = ['cu', 'al', 'zn', 'pb', 'ni', 'sn']
precious_metal_mkts = ['au', 'ag']
ind_metal_mkts = ferrous_products_mkts + ferrous_mixed_mkts + base_metal_mkts  
petro_chem_mkts = ['l', 'pp', 'v', 'TA', 'MA', 'bu'] #, 'sc', 'fu', 'eg']
ind_all_mkts = ind_metal_mkts + petro_chem_mkts
ags_oil_mkts = ['m', 'RM', 'y', 'p', 'OI', 'a', 'c', 'cs'] #, 'b']
ags_soft_mkts = ['CF', 'SR', 'jd']#, 'AP', 'sp']
ags_all_mkts = ags_oil_mkts + ags_soft_mkts
eq_fut_mkts = ['IF', 'IH', 'IC']
bond_fut_mkts = ['T', 'TF']
fin_all_mkts = eq_fut_mkts + bond_fut_mkts
commod_all_mkts = ind_all_mkts + ags_all_mkts + precious_metal_mkts
all_markets = commod_all_mkts + fin_all_mkts


In [5]:
dbconfig = dbaccess.hist_dbconfig
func = dbaccess.mysql_replace_into
cnx = dbaccess.connect(**dbconfig)
conn = create_engine('mysql+mysqlconnector://{user}:{passwd}@{host}/{dbase}'.format( \
                    user = dbconfig['user'], \
                    passwd = dbconfig['password'],\
                    host = dbconfig['host'],\
                    dbase = dbconfig['database']), echo=False)
for exch in ['CZCE', 'DCE', 'SHFE', 'CFFEX', 'INE']:
    for prod in misc.product_code[exch]:
        if ('_' in prod):
            continue
        df = pd.read_sql("select * from fut_min where instID like '{prodcode}' and exch = 'NA';".format(prodcode = prod+'%'), cnx)
        print(exch, prod, len(df))
        if len(df) > 0:
            cursor = cnx.cursor()
            try:
                cursor.execute("delete from fut_min where instID like '{prodcode}' and exch = 'NA';".format(prodcode = prod+'%'))
                cnx.commit()
            except:
                cnx.rollback()
        
            df['exch'] = exch
            print(df[-3:])
            df.to_sql('fut_min', con = conn, if_exists='append', index=False, method = func)


CZCE ER 0
CZCE RO 0
CZCE WS 0
CZCE WT 0
CZCE WH 0
CZCE PM 0
CZCE CF 0
CZCE CY 0
CZCE SR 0
CZCE TA 0
CZCE OI 0
CZCE RI 0
CZCE ME 0
CZCE FG 0
CZCE RS 0
CZCE RM 0
CZCE TC 0
CZCE JR 0
CZCE LR 0
CZCE MA 0
CZCE SM 0
CZCE SF 0
CZCE ZC 0
CZCE AP 259183
       instID  exch            datetime     open     high      low    close  \
259180  AP907  CZCE 2018-09-20 14:57:00  12410.0  12427.0  12410.0  12427.0   
259181  AP907  CZCE 2018-09-20 14:58:00  12427.0  12427.0  12427.0  12427.0   
259182  AP907  CZCE 2018-09-20 14:59:00  12427.0  12427.0  12427.0  12427.0   

        volume  openInterest        date  min_id  
259180     4.0       13028.0  2018-09-20    2057  
259181     0.0       13028.0  2018-09-20    2058  
259182     0.0       13028.0  2018-09-20    2059  
CZCE CJ 898
    instID  exch            datetime    open    high     low   close  volume  \
895  CJ912  CZCE 2019-04-30 14:57:00  8835.0  8845.0  8835.0  8840.0  2716.0   
896  CJ912  CZCE 2019-04-30 14:58:00  8840.0  8840.0  8825.0  

In [8]:
field_list = ['open', 'high', 'low', 'close', 'volume', 'openInterest', 'contract', 'shift']

start_date = datetime.date(2014,4,1)
end_date = datetime.date(2020,7,6)

sim_products = ['AP'] #['cu', 'al', 'zn', 'pb', 'ni', 'sn'] # ['ZC', 'jm', 'ru', 'FG'] # 
need_shift = 1
freq = 'd'

args = {'n': 1, 'roll_rule': '-35b', 'freq': freq, 'need_shift': need_shift}
ferrous_products_args = args
ferro_mixed_mkt_args = args
base_args = {'n': 1, 'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift}
ni_args = {'n': 1, 'roll_rule': '-40b', 'freq': freq, 'need_shift': need_shift}
eq_args = {'n': 1, 'roll_rule': '-1b', 'freq': freq, 'need_shift': need_shift}
bond_args = {'n': 1, 'roll_rule': '-20b', 'freq': freq, 'need_shift': need_shift}
precious_args = {'n': 1, 'roll_rule': '-25b', 'freq': freq, 'need_shift': need_shift}

df_list = []
for idx, asset in enumerate(sim_products):
    use_args = args
    if asset in eq_fut_mkts:
        use_args = eq_args
    elif asset in ['cu', 'al', 'zn', 'pb', 'sn']:
        use_args = base_args
    elif asset in ['ni']:
        use_args = ni_args
    elif asset in bond_fut_mkts:
        use_args = bond_args
    elif asset in precious_metal_mkts:
        use_args = precious_args
    use_args['start_date'] = max(backtest.sim_start_dict[asset], start_date)
    use_args['end_date'] = end_date
    print("loading mkt = %s, args = %s" % (asset, use_args))
    df = misc.nearby(asset, **use_args)
    if freq == 'm':
        df = misc.cleanup_mindata(df, asset)
    if (idx == 0) and (freq == 'm'):
        df_list.append(df[['date', 'min_id']])
    xdf = df[field_list]
    xdf.columns = [(asset, col) for col in xdf.columns]
    df_list.append(xdf)



loading mkt = AP, args = {'n': 1, 'roll_rule': '-35b', 'freq': 'd', 'need_shift': 1, 'start_date': datetime.date(2018, 4, 1), 'end_date': datetime.date(2020, 7, 6)}


In [11]:
import collections
print([item for item, count in collections.Counter(df_list[0].index).items() if count > 1])
all_df = pd.concat(df_list, axis = 1, sort = False).sort_values(['date', 'min_id']).dropna()
print(all_df)


[datetime.date(2018, 5, 21), datetime.date(2018, 5, 23), datetime.date(2018, 5, 24), datetime.date(2018, 5, 25), datetime.date(2018, 5, 28), datetime.date(2018, 5, 29), datetime.date(2018, 5, 30), datetime.date(2018, 5, 31), datetime.date(2018, 6, 1), datetime.date(2018, 6, 4), datetime.date(2018, 6, 5), datetime.date(2018, 6, 6), datetime.date(2018, 6, 7), datetime.date(2018, 6, 8), datetime.date(2018, 6, 11), datetime.date(2018, 6, 12), datetime.date(2018, 6, 13), datetime.date(2018, 6, 14), datetime.date(2018, 6, 15), datetime.date(2018, 6, 19), datetime.date(2018, 6, 20), datetime.date(2018, 6, 21), datetime.date(2018, 6, 22), datetime.date(2018, 6, 25), datetime.date(2018, 6, 26), datetime.date(2018, 6, 27), datetime.date(2018, 6, 28), datetime.date(2018, 6, 29), datetime.date(2018, 7, 2), datetime.date(2018, 7, 3), datetime.date(2018, 7, 4), datetime.date(2018, 7, 5), datetime.date(2018, 7, 9), datetime.date(2018, 7, 10), datetime.date(2018, 7, 11), datetime.date(2018, 7, 12), da

KeyError: 'min_id'